# TypeCtx
We will first take a script and add the Cassette logic to get the information from the stack trace.

In [1]:
using Cassette;
using DifferentialEquations: ODEProblem, solve

In [2]:
function main()
    
    # define our ode
    function sir_ode(du, u, p, t)  
        #Infected per-Capita Rate
        β = p[1]
        #Recover per-capita rate
        γ = p[2]
        #Susceptible Individuals
        S = u[1]
        #Infected by Infected Individuals
        I = u[2]

        du[1] = -β * S * I
        du[2] = β * S * I - γ * I
        du[3] = γ * I
    end

    #Pram = (Infected Per Capita Rate, Recover Per Capita Rate)
    pram = [0.1,0.05]
    #Initial Prams = (Susceptible Individuals, Infected by Infected Individuals)
    init = [0.99,0.01,0.0]
    tspan = (0.0,200.0)
    
    # create a var to our problem
    sir_prob = ODEProblem(sir_ode, init, tspan, pram)
    solution = solve(sir_prob)
    
end

main (generic function with 1 method)

In [3]:
mutable struct FCollector{I,F,C}
    depth::I
    frame::F
    data::Vector{C}
end

In [4]:
function FCollector(d::Int, f)
    FCollector(d, f, FCollector[])
end

FCollector

In [21]:
""" 
Frame(func, args, ret, subtrace)

a structure to hold metadata for recursive type information
"""
mutable struct Frame{F,T,U}
    func::F
    args::T
    ret::U
end

Frame

In [6]:
# define the 
Cassette.@context TypeCtx;

In [7]:
maxdepth =3
extractor = FCollector(maxdepth, Frame(nothing, (), nothing,))

FCollector{Int64,Frame{Nothing,Tuple{},Nothing},FCollector}(3, Frame{Nothing,Tuple{},Nothing}(nothing, (), nothing), FCollector[])

In [8]:
ctx = TypeCtx(metadata = extractor);

In [9]:
# add boilerplate for functionality
function Cassette.overdub(ctx::TypeCtx, f, args...)
    c = FCollector(ctx.metadata.depth-1, Frame(f, args, Any))
    push!(ctx.metadata.data, c)
    if c.depth > 0 && Cassette.canrecurse(ctx, f, args...)
        newctx = Cassette.similarcontext(ctx, metadata = c)
        z = Cassette.recurse(newctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    else
        z = Cassette.fallback(ctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    end
end

In [10]:
Cassette.canrecurse(ctx::TypeCtx,::typeof(ODEProblem), args...) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Base.vect), args...) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(FCollector)) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Frame)) = false

In [67]:
Cassette.overdub(ctx,main);

# Create a graph

In [12]:
using LightGraphs;
using MetaGraphs;

In [74]:
g = MetaDiGraph()
function build_graph(collector::FCollector)
    add_vertex!(g,:name,collector.frame.args)
    add_vertex!(g,:name,collector.frame.ret)
    add_edge!(g,nv(g)-1,nv(g),:name,collector.frame.func)
    for frame in collector.data
        build_graph(frame)
    end
    return g
end

build_graph (generic function with 1 method)

In [75]:
build_graph(extractor)

{42, 21} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)